In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [83]:
df = pd.read_csv("phone_user_review_file_1.csv", sep=',', encoding='utf-8', encoding_errors='ignore')

In [86]:
import pandas as pd

# Örnek veri setinizi yükleyin
# df = pd.read_csv('your_dataset.csv')  # Veri setinizi buradan yükleyebilirsiniz

# Geçerli modelleri listeleyelim
valid_models = [
    '/cellphones/samsung-galaxy-s5/',
    '/cellphones/samsung-galaxy-s7-edge/',
    '/cellphones/samsung-galaxy-s6/',
    '/cellphones/samsung-galaxy-s7-789999/',
    '/cellphones/samsung-galaxy-s6-edge-sm-g925f/'
]

# 'lang' sütununda 'en' olanları ve 'phone_url' sütunundaki geçerli modelleri filtreleyelim
filtered_df = df[df['lang'] == 'en']
filtered_df = filtered_df[filtered_df['phone_url'].isin(valid_models)]

# Telefon model adlarını URL'den çekelim
def extract_model_name(url):
    if 'samsung-galaxy-s6-edge' in url:
        return 's6 edge'  # 's6 edge' için özel kontrol
    elif 'samsung-galaxy-s5' in url:
        return 's5'
    elif 'samsung-galaxy-s7-edge' in url:
        return 's7 edge'
    elif 'samsung-galaxy-s6' in url:
        return 's6'
    elif 'samsung-galaxy-s7' in url:
        return 's7'

# URL'den model adını çıkartalım
filtered_df['phone'] = filtered_df['phone_url'].apply(extract_model_name)

# Gerekli sütunları seçelim ve adlarını değiştirelim
cleaned_df = filtered_df[['phone', 'extract', 'score']].copy()
cleaned_df.rename(columns={'extract': 'review', 'score': 'score'}, inplace=True)

# İndeksi sıfırlayalım
cleaned_df.reset_index(drop=True, inplace=True)

# Sonuçları df'ye kaydedelim
df = cleaned_df

In [87]:
# Yeni df'yi görüntüleyelim
print(df["phone"].value_counts())


phone
s5         9667
s7 edge    8925
s6         8686
s7         8209
s6 edge    4380
Name: count, dtype: int64


In [6]:
def categorize_score(score):
    if score <= 3:
        return 'Negative'
    elif 4 <= score <= 6:
        return 'Neutral'
    elif 7 <= score <= 10:
        return 'Positive'

In [7]:
df['score'] = df['score'].apply(categorize_score)


In [88]:
df = df.dropna(subset=["score"])

In [89]:
df = df.dropna(subset=['review'])

In [91]:
df["score"].value_counts()

,count
score,
10.0,20702
9.6,3666
8.0,3518
9.2,2543
2.0,2397
8.8,1672
8.4,1265
6.0,1184
4.0,839


In [9]:
df["score"]

,score
0,Positive
1,Positive
2,Positive
3,Positive
4,Positive
...,...
39862,Positive
39863,Positive
39864,Positive
39865,Positive


In [ ]:
df = df.dropna(subset=['processed_review'])


In [10]:
df["score"].isna().sum()

0

In [11]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Veri setindeki sınıf etiketlerini alın
y_train = df["score"]

# Benzersiz sınıf etiketlerini alın
classes = np.unique(y_train)

# Sınıf ağırlıklarını hesaplayın
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)

# Ağırlıkları bir sözlük olarak görmek için
class_weights_dict = dict(zip(classes, class_weights))
print(class_weights_dict)


{'Negative': 5.357560909214645, 'Neutral': 5.709602552944589, 'Positive': 0.37904569310029373}


In [12]:
df["score"].value_counts()

,count
score,
Positive,34615
Negative,2449
Neutral,2298


In [13]:
df.head()

,phone,review,score
0,s7 edge,I like how this phone works I had a note 7. Al...,Positive
1,s7 edge,Love the screen size and battery life. I get t...,Positive
2,s7 edge,I absolutely love this phone.It's easy to hold...,Positive
3,s7 edge,The good: - Stunning display. Impressive 5.5 q...,Positive
4,s7 edge,I would definitely recommend this phone to any...,Positive


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39362 entries, 0 to 39866
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   phone   39362 non-null  object
 1   review  39165 non-null  object
 2   score   39362 non-null  object
 3   emoji   39362 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [24]:
import spacy
import re
from tqdm import tqdm  # tqdm'i import ediyoruz

spacy.prefer_gpu()

# spaCy İngilizce modelini yükle
nlp = spacy.load('en_core_web_sm')

# Veri ön işleme fonksiyonu
def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r"[^\w\s]", "", text)

    # Küçük harfe çevirme
    text = text.lower()

    # spaCy ile metni işleyin (tokenization, stop word temizleme, lemmatization)
    doc = nlp(text)

    # Stop words'leri temizleme ve lemmatization (kök haline getirme)
    words = [token.lemma_ for token in doc if not token.is_stop]

    # İşlenmiş metni birleştirme
    return " ".join(words)

# tqdm ile ilerleme çubuğunu ekleyelim
tqdm.pandas()  # pandas'la uyumlu hale getiriyoruz

# Veri setini işleme
processed_data = df["review"].progress_apply(preprocess_text)

# Sonuçları görüntüleme
print(processed_data)


100%|██████████| 39165/39165 [12:29<00:00, 52.28it/s]

0        like phone work note 7 like note 7 stylus ipho...
1        love screen size battery life day moderate use...
2        absolutely love phoneit easy hold screen simpl...
3        good   stunning display impressive 55 quad hd ...
4        definitely recommend phone look upgrade s6s6 e...
                               ...                        
39862    preordere phone day excited look like lot thou...
39863    highly recommend phone new smart phone world b...
39864    galaxy s3 actually galaxy s4 great device sens...
39865    not wrong galaxy phonethe galaxy s5 late great...
39866    samsung fan own galaxy s2 s3 s4 ve galaxy s5 4...
Name: review, Length: 39165, dtype: object


In [ ]:
df["processed_data"]=processed_data

In [102]:
processed_data.head()


,review
0,like phone work note like note stylus ipho...
1,love screen size battery life day moderate use...
2,absolutely love phoneit easy hold screen simpl...
3,good stunning display impressive quad hd s...
4,definitely recommend phone look upgrade ss edg...


In [60]:
df3.head()

,phone,review,score,emoji,processed_review
0,s7 edge,I like how this phone works I had a note 7. Al...,Positive,0,like phone work note 7 like note 7 stylus ipho...
1,s7 edge,Love the screen size and battery life. I get t...,Positive,0,love screen size battery life day moderate use...
2,s7 edge,I absolutely love this phone.It's easy to hold...,Positive,0,absolutely love phoneit easy hold screen simpl...
3,s7 edge,The good: - Stunning display. Impressive 5.5 q...,Positive,0,good stunning display impressive 55 quad hd ...
4,s7 edge,I would definitely recommend this phone to any...,Positive,0,definitely recommend phone look upgrade s6s6 e...


In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# TF-IDF Vectorizer ayarları
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')

# Veriyi vektörize et
X_tfidf = tfidf.fit_transform(df['processed_review'])

# Hedef değişkeni (labels) alıyoruz
y = df['score']

# Veriyi eğitim ve test setlerine ayırıyoruz (%80 eğitim, %20 test)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Modeli oluşturuyoruz
model = LogisticRegression(class_weight=class_weights_dict)

# Modeli eğitim verisiyle eğitiyoruz
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yapıyoruz
y_pred = model.predict(X_test)

# Modelin performansını değerlendiriyoruz
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Negative       0.42      0.69      0.52       481
     Neutral       0.22      0.48      0.30       458
    Positive       0.98      0.86      0.91      6894

    accuracy                           0.82      7833
   macro avg       0.54      0.67      0.58      7833
weighted avg       0.90      0.82      0.85      7833



In [38]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['score'], test_size=0.2, random_state=42)

# TF-IDF Vectorizer ayarları
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')

# Veriyi vektörize et (Eğitim seti için)
X_train_tfidf = tfidf.fit_transform(X_train)

# Test seti için dönüşümü uygulayın
X_test_tfidf = tfidf.transform(X_test)

# Hiperparametre aralığı tanımla
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization parametresi
    'solver': ['lbfgs', 'liblinear', 'saga'],  # Çözümleyici seçenekleri
    'max_iter': [100, 200, 300],  # Maksimum iterasyon sayısı
    'class_weight': [class_weights_dict, 'balanced', None],  # Sınıf ağırlıkları
}

# Logistic Regression modelini oluştur
log_reg = LogisticRegression()

# GridSearchCV ile parametre optimizasyonu yap
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Modeli eğit
grid_search.fit(X_train_tfidf, y_train)

# En iyi parametreleri ve skoru yazdır
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_}")

# Test verisi üzerinde en iyi modelle tahmin yapalım
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)

# Sonuçları değerlendirelim
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 135 candidates, totalling 675 fits
Best Parameters: {'C': 1, 'class_weight': None, 'max_iter': 100, 'solver': 'lbfgs'}
Best Cross-Validation Score: 0.9039320816291274
              precision    recall  f1-score   support

    Negative       0.73      0.50      0.59       485
     Neutral       0.36      0.09      0.15       446
    Positive       0.93      0.99      0.96      6902

    accuracy                           0.91      7833
   macro avg       0.67      0.53      0.57      7833
weighted avg       0.88      0.91      0.89      7833



In [78]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(df['processed_review'], df['score'], test_size=0.2, random_state=42)

# TF-IDF Vectorizer ayarları
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')

# Veriyi vektörize et (Eğitim seti için)
X_train_tfidf = tfidf.fit_transform(X_train)

# Test seti için dönüşümü uygulayın
X_test_tfidf = tfidf.transform(X_test)

# Hiperparametre aralığı tanımla
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization parametresi
    'solver': ['lbfgs', 'liblinear', 'saga'],  # Çözümleyici seçenekleri
    'max_iter': [100, 200, 300],  # Maksimum iterasyon sayısı
    'class_weight': ['balanced', None],  # Sınıf ağırlıkları
}

# Logistic Regression modelini oluştur
log_reg = LogisticRegression()

# GridSearchCV ile parametre optimizasyonu yap
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Modeli eğit
grid_search.fit(X_train_tfidf, y_train)

# En iyi parametreleri ve skoru yazdır
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_}")

# Test verisi üzerinde en iyi modelle tahmin yapalım
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)

# Sonuçları değerlendirelim
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best Parameters: {'C': 1, 'class_weight': None, 'max_iter': 200, 'solver': 'saga'}
Best Cross-Validation Score: 0.9026429617115539
              precision    recall  f1-score   support

    Negative       0.67      0.48      0.56       481
     Neutral       0.40      0.08      0.14       458
    Positive       0.92      0.99      0.96      6894

    accuracy                           0.91      7833
   macro avg       0.67      0.52      0.55      7833
weighted avg       0.88      0.91      0.88      7833

